# Split the basin example base model

## Notebook Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib as pl

import flopy
from flopy.discretization import StructuredGrid
from flopy.mf6.utils import Mf6Splitter

In [ ]:
# import containerized functionality from defaults.py
from defaults import *

### Parallel settings

1. Set voronoi to `True` to process Voronoi grid base and parallel models.
2. Set the number of models in the row and column directions. NOTE: Set nrow_blocks or ncol_blocks to 0 to load a model split with metis or if a Voronoi grid model.

In [ ]:
voronoi = False
nrow_blocks, ncol_blocks = 8, 0

In [ ]:
base_ws, parallel_ws = get_workspaces(nrow_blocks, ncol_blocks, voronoi=voronoi)
base_ws, parallel_ws

### Load the base basin model 

In [ ]:
base_sim = flopy.mf6.MFSimulation.load(
    sim_name="basin", 
    sim_ws=base_ws,
)

In [ ]:
base_gwf = base_sim.get_model()

### Load the parallel basin model

In [ ]:
new_sim = flopy.mf6.MFSimulation.load(
    sim_name="basin", 
    sim_ws=parallel_ws,
    verbosity_level=0,
)

#### Load the node mapping array for the parallel basin model

In [ ]:
json_path = parallel_ws / "mfsplit_node_mapping.json"
mfsplit = Mf6Splitter(base_sim)
mfsplit.load_node_mapping(new_sim, json_path)

### Plot the multi-model and single model heads

In [ ]:
base_head = np.squeeze(base_gwf.output.head().get_data())
hmin, hmax = (
    base_head.min(),
    np.where(base_head < 1e30, base_head, 0).max(),
)
contours = np.arange(0, 100, 10)

#### Build a dictionary with the model heads for each partition

In [ ]:
model_names = list(new_sim.model_names)
head_dict = {}
for modelname in model_names:
    mnum = int(modelname.split("_")[-1])
    head = new_sim.get_model(modelname).output.head().get_data()
    head_dict[mnum] = head

#### Build a single head array

In [ ]:
new_head = mfsplit.reconstruct_array(head_dict)

#### Plot the results

In [ ]:
fig = plt.figure(figsize=(figwidth, figheight * 1.3333))
t = new_head - base_head
hv = [new_head, base_head, t]
titles = ["Multiple models", "Single model", "Multiple - single"]
for idx in range(3):
    ax = fig.add_subplot(3, 1, idx + 1)
    ax.set_aspect("equal")
    ax.set_title(titles[idx])

    if idx < 2:
        levels = contours
        vmin = hmin
        vmax = hmax
    else:
        levels = None
        vmin = None
        vmax = None

    pmv = flopy.plot.PlotMapView(model=base_gwf, ax=ax, layer=0)
    h = pmv.plot_array(hv[idx], vmin=vmin, vmax=vmax)
    if levels is not None:
        c = pmv.contour_array(
            hv[idx],
            levels=levels,
            colors="white",
            linewidths=0.75,
            linestyles=":",
        )
        plt.clabel(c, fontsize=8)
    if not voronoi:
        pmv.plot_inactive()
    plt.colorbar(h, ax=ax, shrink=0.5)

plt.show()